In [1]:
import pandas as pd

Subir el archivo de navegacion a Colab

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving navegacion.csv to navegacion.csv
User uploaded file "navegacion.csv" with length 3384875 bytes


Crear el dataframe a partir del dataset de navegacion

In [ ]:
dfNavegacion = pd.read_csv("navegacion.csv",sep=";")
print(dfNavegacion.head(10))

<h1>Analisis Exploratorio de Datos (EDA)</h1>
<h2>Limpia de datos </h2>
<p>Procedemos a eliminar todos los datos que no sean coherentes, ya que afectaria al resultado final</p>

In [ ]:
# Comprobar cuantos id_user hay repetidos
print(dfNavegacion['id_user'].value_counts().head(50))

In [9]:
# Comprobar cuantos gclid hay repetidos
print(dfNavegacion['gclid'].value_counts().head(50))

EAIaIQobChMIwPu5t4qs3AIVAQAAAB0BAAAAEAAYACAAEgJVzfD_BwE                                         124
CjwKCAjwjbCDBhAwEiwAiudBy3jlJeW4vAzqieU59RXrhxtMvbHXgK5UdK0KHtUCtcp6GWd38D0u6BoC76EQAvD_BwE      10
Cj0KCQjw-NaJBhDsARIsAAja6dOww1IgSWThNMRCIYMOqCLia0Udue7wdypptdIMPRlUBCr6gTi-_C4aAgdIEALw_wcB      5
Cj0KCQjw-NaJBhDsARIsAAja6dMELUkx0xI6udIqEYLUIQSpbP0FQtNk2aVxcUiMQUjb058AhqTsvhcaAho4EALw_wcB      5
Cj0KCQjw-NaJBhDsARIsAAja6dOmcKjgn77p5hHT_5b1XIyg6fHlpWRs8rdXlMlnlzQsJezWS4JyGQYaAv7DEALw_wcB      4
Cj0KCQjw-NaJBhDsARIsAAja6dOTN1Pp0Ow-6VLYGiyJk0lpjmIhfm?r5nZ_4TBTz476oruy13Awa0AaAtS7EALw_wcB      4
Cj0KCQjw-NaJBhDsARIsAAja6dPLhRHDqi-qYz4dcE9XFVvOcq4o0eiCW5ss8t3T_h7u9VB4NXAS-FkaAhKzEALw_wcB      4
Cj0KCQjw-NaJBhDsARIsAAja6dOMdiySET1yjbRJgb3qi8RdOv3ETY-RRl1FLazS7VULGSOxx18mEVUaAhcea?Lw_wcB      4
Cj0KCQjw-NaJBhDsARIsAAja6dPDldGfiETrgOJl8913cSBMnZ30bWT4i0PvY9XYWSF39CF__gMmYqcaAhWsEALw_wcB      4
Cj0KCQjw-NaJBhDsARIsAAja6dPF-dZH3B6kU3Lp_q8fn9f8bl4tz8uOoZbmJJcL6EkgWPG6mvqGRZ0aAoWoEALw_wcB      4


In [ ]:
# Comprobar si los gclid corresponden a los mismos usuarios
print(dfNavegacion.groupby(['gclid','id_user'])['ts'].count()   )

In [8]:
# Hemos visto que el gclid=0 pertenece a diferentes usuarios, por lo
# que no es un dato coherente y lo elimino
gclid0 = dfNavegacion[dfNavegacion['gclid'] == '0'].index
print(gclid0)
dfNavegacion = dfNavegacion.drop(gclid0)

Int64Index([ 129,  139,  356,  764, 1132, 1135, 1403, 1847, 1915, 2022, 2030,
            2216, 2267, 2377, 2590, 2598, 2812, 3122, 3164, 3238, 3276, 3281,
            3298, 3358, 3772, 3956, 4181, 4491, 4512, 4775, 5264, 7064, 7074,
            7102, 7134],
           dtype='int64')


In [11]:
# tambien vamos a borrar el gclid que tenia 124 ocurrencias
gclid124 = dfNavegacion[dfNavegacion['gclid'] == 'EAIaIQobChMIwPu5t4qs3AIVAQAAAB0BAAAAEAAYACAAEgJVzfD_BwE'].index
print(gclid124)
dfNavegacion = dfNavegacion.drop(gclid124)

Int64Index([   6,    7,   34,   41,   47,   49,   50,   52,   53,   60,
            ...
            7231, 7245, 7246, 7247, 7252, 7257, 7258, 7259, 7263, 7265],
           dtype='int64', length=124)


<h1>Empezamos a procesar la URL que tengo datos que luego necesitare</h1>
<h2>separar los datos url_landing en columnas</h2>
<p> hay que asegurarse que no este vacia</p>

In [13]:
# Me quedo solo con las url_landing que no sean nulas
serieUrlLanding = dfNavegacion['url_landing'][dfNavegacion['url_landing'].notnull()]
dfNavegacion[['url_base','idUser','uuid_url','camp','adg','device','sl','adv','rec','vacia']] = serieUrlLanding.str.split(pat='&',n=10,expand=True)
print(dfNavegacion.columns)

Index(['ts', 'uuid', 'id_user', 'gclid', 'user_recurrent', 'url_landing',
       'url_base', 'idUser', 'uuid_url', 'camp', 'adg', 'device', 'sl', 'adv',
       'rec', 'vacia'],
      dtype='object')


<h2>Eliminar las columnas que no voy a necesitar</h2>

In [15]:
dfNavegacion = dfNavegacion.drop(['idUser','uuid_url','rec', 'vacia'], axis=1)

<h1>Obtener el modelo de coche que viene en la url</h1>

In [ ]:
#print(dfNavegacion['url_base'].head(50))
#url/modelo?parametros
# la parte de la url que me interesa es lo que esta antes de la ?
dfNavegacion['url_base'] = dfNavegacion['url_base'].apply(lambda url: str(url).split('?')[0])
print(dfNavegacion['url_base'].head(50))

In [ ]:
# obtener el modelo de coche y crear una nueva columna
def obtenerModelo(url):
    posicion = url.rfind('/')
    modeloObtenido = url[posicion+1:]
    if modeloObtenido == '' or modeloObtenido == 'renting':
        modeloObtenido = 'home'
    return modeloObtenido

dfNavegacion.insert(loc=7, column='modelo', value=dfNavegacion['url_base'].apply(obtenerModelo))
print(dfNavegacion['modelo'].head(50))

<h1>Crear un orden de usuarios</h1>
<p>Como tenemos usuarios repetidos me daria problema al hacer el merge con el dataframe de conversiones </p>
<p>Voy a crear una columna llamada orden que almacene el orden de los id_user</p>
<p>Una vez que tenga los datos de orden, filtro el dfNavegacion para quedarme solo con el primero (orden=0)</p>

In [20]:
dfNavegacion = dfNavegacion.sort_values(['id_user','ts'])
dfNavegacion.insert(loc=1,column='orden', value=dfNavegacion.groupby('id_user')['ts'].cumcount())

#Filtrar por orden=0
dfNavegacion = dfNavegacion[dfNavegacion['orden']==0]

<h1>Unir el fichero de navegacion y el fichero de conversiones con un merge</h1>
<h2>Primero cargar en colab el dataset de conversiones y generar dfConversiones</h2>

In [21]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving conversiones.csv to conversiones.csv
User uploaded file "conversiones.csv" with length 3165 bytes


In [ ]:
dfConversiones = pd.read_csv("conversiones.csv",sep=";")
print(dfConversiones.head(10))

<h1>Vamos a hacer el merge con el modelo outer para no perder ningun dato</h1>

In [23]:
dfFinal = dfNavegacion.merge(dfConversiones, how='outer', on=['id_user'])

<h2>Crear una nueva columna llamada conversiones con el valor 1 (si es conversion), 0 (si no lo es)</h2>

In [24]:
def esConversion(dato):
  lista = ['CALL','FORM']
  if dato in lista:
    return 1
  else:
    return 0

dfFinal.insert(loc=1, column='conversiones', value=dfFinal['lead_type'].apply(esConversion))

<h1> **************************   Preguntas a responder  *******************</h1>
<h2>Pregunta 1: Cuantas visitas recibe en el dia el cliente?</h2>

In [26]:
#print(dfFinal.head(20))

# crear una columna dia con la fecha de la columna ts
dfFinal.insert(loc=1, column='dia', value=dfFinal['ts'].apply(lambda ts: str(ts).split(' ')[0]))
print(dfFinal.groupby('dia')['dia'].count())

dia
2021-09-06    5640
2021-09-07       4
nan             13
Name: dia, dtype: int64


<h3>Cuantas de ellas convierten y cuantas no (en %)</h3>

In [27]:
# Filtrar por 2021-09-06
visitas_dia_6_septiembre = dfFinal[dfFinal['dia'] == '2021-09-06']
num_visitas_dia = visitas_dia_6_septiembre['dia'].count()

# cuento las visitas con conversion
visitas_convierten = visitas_dia_6_septiembre[visitas_dia_6_septiembre['conversiones']==1]['ts'].count()

# cuento las visitas sin conversion
visitas_no_convierten = visitas_dia_6_septiembre[visitas_dia_6_septiembre['conversiones']==0]['ts'].count()

# calculo los porcentajes
porcentaje_visitas_convierten = round( (visitas_convierten / num_visitas_dia) * 100, 2)
porcentaje_visitas_no_convierten = round( (visitas_no_convierten / num_visitas_dia) * 100, 2)
print("Convierten:", porcentaje_visitas_convierten)
print("No convierten:", porcentaje_visitas_no_convierten)

Convierten: 0.27
No convierten: 99.73


<h2>Pregunta 2: Por tipo de conversion (CALL o FORM), cuantas hay de cada una?</h2>

In [28]:
print(dfFinal.groupby('lead_type')['lead_type'].count())

lead_type
CALL    19
FORM     9
Name: lead_type, dtype: int64


<h2>Pregunta 3: Porcentaje de usuarios recurrentes sobre el total de usuarios</h2>

In [30]:
# Contar cuantos usuarios tenemos
total_usuarios = dfFinal['id_user'].count()

# Contar los usuarios recurrentes
usuarios_recurrentes = dfFinal[dfFinal.user_recurrent & dfFinal.user_recurrent.notnull()]['ts'].count()

# calculamos el porcentaje
porcentaje = round((usuarios_recurrentes / total_usuarios) * 100 , 2)
print("Usuarios recurrentes: ", porcentaje)

Usuarios recurrentes:  18.7


<h2>Pregunta 4: Coche mas visitado</h2>

In [ ]:
print(dfFinal['modelo'].value_counts())

<h3>Es el que mas convierte?</h3>

In [33]:
# Voy a filtrar el dataset para tener solo los registros que han convertido y 
# luego los agrupo por modelo
print(dfFinal[dfFinal['conversiones'] == 1].groupby('modelo')['conversiones'].count())

modelo
cea-electrico    1
clin200          2
clin400          3
dep30            1
home             5
ixs-electrico    1
tria             2
Name: conversiones, dtype: int64


<h1>Exportar el dfFinal a un Excel para elaborar informes, graficos, histogramas, ...etc</h2>

In [35]:
dfFinal.to_excel("dataframe_final.xlsx", index=False)